#Бибилиотеки

In [ ]:
#Установим необходимые библиотеки

!pip install scikit-surprise

!pip install implicit

!pip install lightfm

In [ ]:
#Загрузим необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import scipy.sparse as sparse
from surprise.model_selection import train_test_split

import implicit
import pickle

from lightfm import LightFM

#Подключимся к диску с данными

In [ ]:
# Если работаем через google disk
# Подключимся к диску с данными
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Директория с датасетом
dir_path ='/content/gdrive/MyDrive/Final_work_3_DS/data_recommender_system'

#LightFM

Воспользуемся библиотекой LightFM

Библиотека требует предварительной предобработки данных датасета.

LightFM работает с рейтингами. Сформируем рейтинги каждого продукта. Самый частый продукт будет с рейтингом 1. Если несколько продуктов покупались одинаковое количество раз то и ранг у них будет одинаковый.

In [ ]:
df_counter = pd.read_csv(dir_path + '/counter_df')

In [ ]:
#Столбец Рангов
df_counter['rank_pr'] = None

In [ ]:
#Берем значения
df_values=df_counter[['user_id','product_id','count']].values

n=1
rank = []
user_n = df_values[0][0]
prod_n = df_values[0][1]
count_n = df_values[0][2]
rank.append(n)

#Проходим по массиву и выставляем ранг
for row in df_values[1:]:

  if row[0] == user_n:
    if row[2] == count_n:
      rank.append(n)
      count_n = row[2]
    else:
      n += 1
      rank.append(n)
      count_n = row[2]
  
  else:
    n=1    

    user_n = row[0]
    prod_n = row[1]
    count_n = row[2]
    rank.append(n)


#Добавляем список рангов в датасет
df_counter.loc[:, 'rank_pr'] = rank

In [ ]:
#перевернем ранги, так как LightFM работаем с максимальным рангом
df_counter.rank_pr = (df_counter.rank_pr.max() +1) - df_counter.rank_pr

In [ ]:
df_counter = df_counter.drop('count', axis = 1)

In [ ]:
#Создадим разряженную матрицу
sparse_item_user = sparse.csr_matrix((df_counter['rank_pr'].astype(float),
                                      (df_counter['product_id'], 
                                       df_counter['user_id']),))

In [ ]:
sparse_user_item = sparse.csr_matrix((df_counter['rank_pr'].astype(float),
                                      (df_counter['user_id'], 
                                       df_counter['product_id']),))

In [ ]:
#Создадим модель
model_fm = LightFM(no_components=50, loss='warp')

In [ ]:
#Обучение
model_fm.fit(sparse_user_item, epochs=100)

In [ ]:
#Сформируем файл и посмотрим на результат
submission_df = pd.DataFrame()
submission_df['user_id'] = df_transact.user_id.unique()
submission_df.index = submission_df['user_id']
submission_df['product_id'] = None

for i in tqdm(submission_df.user_id.unique()):
  prediction = model_fm.predict(int(i), np.arange(sparse_user_item.shape[1]))
  top_items = np.argsort(-prediction)
  top_items = top_items[:10].tolist()
  prediction = [str(x) for x in top_items]
  submission_df.at[i, 'product_id'] = " ".join(prediction)

100%|██████████| 100000/100000 [23:06<00:00, 72.15it/s]


In [ ]:
#Сохраним итоговые рекомендации на диск
submission_df.to_csv(dir_path + '/submission.csv', sep=',', index=False)

MAP@10

**Score: 0.05161** (epochs=100, no_components = 10)

**Score: 0.05341** (epochs=100, no_components = 20)

**Score: 0.05428** (epochs=100, no_components = 50)